In [ ]:
import polars as pl
from datetime import datetime

In [ ]:
csv_files = {
    "user_table.csv": "user_table.parquet",
    "home_page_table.csv": "home_page_table.parquet",
    "search_page_table.csv": "search_page_table.parquet",
    "payment_page_table.csv": "payment_page_table.parquet",
    "payment_confirmation_table.csv": "payment_confirmation_table.parquet"
}

for csv_name, parquet_name in csv_files.items():
    print(f"Конвертируем {csv_name} -> {parquet_name}")
    df = pl.read_csv(f"/content/{csv_name}")
    df.write_parquet(f"/content/{parquet_name}")

Конвертируем user_table.csv -> user_table.parquet
Конвертируем home_page_table.csv -> home_page_table.parquet
Конвертируем search_page_table.csv -> search_page_table.parquet
Конвертируем payment_page_table.csv -> payment_page_table.parquet
Конвертируем payment_confirmation_table.csv -> payment_confirmation_table.parquet


In [ ]:
df = (
    pl.scan_parquet("/content/user_table.parquet")
    .select(["user_id", "date", "device", "sex"])

    .join(
        pl.scan_parquet("/content/home_page_table.parquet").select(["user_id", "page"]),
        on="user_id",
        how="left"
    ).rename({"page": "Home page"})

    .join(
        pl.scan_parquet("/content/search_page_table.parquet").select(["user_id", "page"]),
        on="user_id",
        how="left"
    ).rename({"page": "Search page"})

    .join(
        pl.scan_parquet("/content/payment_page_table.parquet").select(["user_id", "page"]),
        on="user_id",
        how="left"
    ).rename({"page": "Payment page"})

    .join(
        pl.scan_parquet("/content/payment_confirmation_table.parquet").select(["user_id", "page"]),
        on="user_id",
        how="left"
    ).rename({"page": "Payment confirmation page"})
)

result = df.collect()
print(result.head())

shape: (5, 8)
┌─────────┬────────────┬─────────┬────────┬───────────┬─────────────┬──────────────┬───────────────┐
│ user_id ┆ date       ┆ device  ┆ sex    ┆ Home page ┆ Search page ┆ Payment page ┆ Payment       │
│ ---     ┆ ---        ┆ ---     ┆ ---    ┆ ---       ┆ ---         ┆ ---          ┆ confirmation  │
│ i64     ┆ str        ┆ str     ┆ str    ┆ str       ┆ str         ┆ str          ┆ page          │
│         ┆            ┆         ┆        ┆           ┆             ┆              ┆ ---           │
│         ┆            ┆         ┆        ┆           ┆             ┆              ┆ str           │
╞═════════╪════════════╪═════════╪════════╪═══════════╪═════════════╪══════════════╪═══════════════╡
│ 450007  ┆ 2015-02-28 ┆ Desktop ┆ Female ┆ home_page ┆ null        ┆ null         ┆ null          │
│ 756838  ┆ 2015-01-13 ┆ Desktop ┆ Male   ┆ home_page ┆ null        ┆ null         ┆ null          │
│ 568983  ┆ 2015-04-09 ┆ Desktop ┆ Male   ┆ home_page ┆ search_page ┆ null   

In [ ]:
#Подсчёт конверсий для воронки
total_users = result.shape[0]
home_users = result.filter(pl.col('Home page').is_not_null()).shape[0]
search_users = result.filter(pl.col('Search page').is_not_null()).shape[0]
payment_users = result.filter(pl.col('Payment page').is_not_null()).shape[0]
payment_conv_users = result.filter(pl.col('Payment confirmation page').is_not_null()).shape[0]

In [ ]:
print(f'На Search Page перешло {round(100 * (search_users / home_users), 2)} % пользователей')
print(f'На страницу с оплатой из оставшихся перешло {round(100 * (payment_users / search_users), 2)} % пользователей')
print(f'Оплатило же покупку из перешедших {round(100 * payment_conv_users / payment_users, 2)} % пользователей')
print(f'Это значит, что всего из зашедших на сайт оплатили {round(100 * payment_conv_users / home_users, 2)} % пользователей')

На Search Page перешло 50.0 % пользователей
На страницу с оплатой из оставшихся перешло 13.34 % пользователей
Оплатило же покупку из перешедших 7.5 % пользователей
Это значит, что всего из зашедших на сайт оплатили 0.5 % пользователей


In [ ]:
#Cohort-analysis на основе используемого устройства
conversion_by_device = (
    result.group_by("device")
    .agg([
        pl.col("user_id").count().alias("total_users"),
        pl.col("Payment confirmation page").is_not_null().sum().alias("paid_users"),
        (pl.col("Payment confirmation page").is_not_null().sum() / pl.col("user_id").count() * 100)
         .round(2).alias("conv_rate")
    ])
    .with_columns(
        pl.format("{}%", pl.col("conv_rate")).alias("conv_rate_with_percent")
    )
)

print(conversion_by_device.drop("conv_rate"))
print('Как видно из анализа, 1% пользователей смартфонов доходит до покупки в то время, как доля пользователей ПК и ноутбуков, дошедших до оплаты, составляет лишь\n 0.25%')
print('\n такая разница может быть вызвана рядом факторов, а именно: дизайн сайта на смартфонах лучше чем на ПК;')
print('\n когда пользователи заходят с ПК, они изначально не настроены совершать покупку, но затнтересованы лишь посмотреть товары')
print('\n также нельзя исключать, что играет роль сложность оплаты с ПК, в то время как со смартфона совершить транзакцию несложно')

shape: (2, 4)
┌─────────┬─────────────┬────────────┬────────────────────────┐
│ device  ┆ total_users ┆ paid_users ┆ conv_rate_with_percent │
│ ---     ┆ ---         ┆ ---        ┆ ---                    │
│ str     ┆ u32         ┆ u32        ┆ str                    │
╞═════════╪═════════════╪════════════╪════════════════════════╡
│ Mobile  ┆ 30200       ┆ 302        ┆ 1.0%                   │
│ Desktop ┆ 60200       ┆ 150        ┆ 0.25%                  │
└─────────┴─────────────┴────────────┴────────────────────────┘
Как видно из анализа, 1% пользователей смартфонов доходит до покупки в то время, как доля пользователей ПК и ноутбуков, дошедших до оплаты, составляет лишь
 0.25%

 такая разница может быть вызвана рядом факторов, а именно: дизайн сайта на смартфонах лучше чем на ПК;

 когда пользователи заходят с ПК, они изначально не настроены совершать покупку, но затнтересованы лишь посмотреть товары

 также нельзя исключать, что играет роль сложность оплаты с ПК, в то время как со 

In [ ]:
import plotly.express as px

stages = ['Зашли на страницу', 'Поиск товара', 'Окно оплаты', 'Оплата совершена']
values = [home_users, search_users, payment_users, payment_conv_users]

fig = px.funnel(
    y = stages,
    x = values,
    title = 'Воронка конверсии',
    labels = {'x': 'Этапы', 'y': 'Кол-во пользователей'}
)

fig.show()

In [ ]:
#Подсчёт пользователей по девайсам на каждом этапе
counts = result.group_by('device').agg([
    pl.col('Home page').count().alias('home_count'),
    pl.col('Search page').count().alias('search_count'),
    pl.col('Payment page').count().alias('pay_count'),
    pl.col('Payment confirmation page').count().alias('pay_conv_count')
])

mobile_counts = counts.filter(pl.col('device') == 'Mobile').row(0, named=True)
desktop_counts = counts.filter(pl.col('device') == 'Desktop').row(0, named=True)

home_mobile = mobile_counts['home_count']
home_desktop = desktop_counts['home_count']

search_mobile = mobile_counts['search_count']
search_desktop = desktop_counts['search_count']

pay_mobile = mobile_counts['pay_count']
pay_desktop = desktop_counts['pay_count']

pay_conv_mobile = mobile_counts['pay_conv_count']
pay_conv_desktop = desktop_counts['pay_conv_count']

stages = ['Home', 'Search', 'Payment', 'Payment Confirmed']
mobile_values = [home_mobile, search_mobile, pay_mobile, pay_conv_mobile]
desktop_values = [home_desktop, search_desktop, pay_desktop, pay_conv_desktop]
print(mobile_values)


[30200, 15100, 3020, 302]


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

#мобильные пользователи (слева)
fig.add_trace(go.Funnel(
    y=stages,
    x=mobile_values,
    name='Mobile',
    marker=dict(color='#FB5939'),
    textinfo="value+percent initial"
))

#десктопные пользователи (справа)
fig.add_trace(go.Funnel(
    y=stages,
    x=desktop_values,
    name='Desktop',
    marker=dict(color='#66B2FF'),
    textinfo="value+percent initial"
))

#настройки графика
fig.update_layout(
    title='Воронка конверсии с разбивкой по устройствам',
    xaxis_title='Количество пользователей',
    yaxis_title='Этапы воронки',
    legend_title='Тип устройства',
    showlegend=True,
    height=500
)

fig.show()